In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import uproot
import scipy.stats as st
from scipy.special import beta
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random
import os
import shutil
from pathlib import Path
from subprocess import check_output

from config import *

random.seed(30)

data_folder = root_folder + 'user.eschanet.allTrees_v2_0_2_signal_1Lbb_skim.root'

In [2]:
root = uproot.open(data_folder)

In [3]:
signal_id = []
count = 0

for n in root.keys():
        if 'nosys' not in str(n).lower():
            n = str(n)
            signal_id.append('_'.join(n.split('Wh_hbb_')[1].split('_')[0:2]))            

In [4]:
unique_name = np.unique(signal_id)

In [5]:
syst = ['JET_JER_EffectiveNP_1',
'JET_JER_EffectiveNP_2',
'JET_JER_EffectiveNP_3',
'JET_JER_EffectiveNP_4',
'JET_JER_EffectiveNP_5',
'JET_JER_EffectiveNP_6',
'JET_JER_EffectiveNP_7',
'JET_JER_EffectiveNP_8',
'JET_JER_EffectiveNP_9',
'JET_JER_EffectiveNP_10',
'JET_JER_EffectiveNP_11',
'JET_JER_EffectiveNP_12restTerm',
'JET_JER_DataVsMC',
'JET_BJES_Response',
'JET_EffectiveNP_Mixed1',
'JET_EffectiveNP_Mixed2',
'JET_EffectiveNP_Mixed3',
'JET_EffectiveNP_Detector1',
'JET_EffectiveNP_Detector2',
'JET_EffectiveNP_Modelling1',
'JET_EffectiveNP_Modelling2',
'JET_EffectiveNP_Modelling3',
'JET_EffectiveNP_Modelling4',
'JET_EffectiveNP_Statistical1',
'JET_EffectiveNP_Statistical2',
'JET_EffectiveNP_Statistical3',
'JET_EffectiveNP_Statistical4',
'JET_EffectiveNP_Statistical5',
'JET_EffectiveNP_Statistical6',
'JET_EtaIntercalibration_Modelling',
'JET_EtaIntercalibration_NonClosure_highE',
'JET_EtaIntercalibration_NonClosure_negEta',
'JET_EtaIntercalibration_NonClosure_posEta',
'JET_EtaIntercalibration_TotalStat',
'JET_Flavor_Composition',
'JET_Flavor_Response',
'JET_Pileup_OffsetMu',
'JET_Pileup_OffsetNPV',
'JET_Pileup_PtTerm',
'JET_Pileup_RhoTopology',
'JET_PunchThrough_MC16',
'JET_SingleParticle_HighPt',
'EG_RESOLUTION_ALL',
'EG_SCALE_ALL',
'EG_SCALE_AF2',
'EL_EFF_ID_TOTAL_1NPCOR_PLUS_UNCOR',
'EL_EFF_Reco_TOTAL_1NPCOR_PLUS_UNCOR',
'EL_EFF_Iso_TOTAL_1NPCOR_PLUS_UNCOR',
'MUON_ID',
'MUON_MS',
'MUON_SCALE',
'MUON_SAGITTA_RHO',
'MUON_SAGITTA_RESBIAS',
'MUON_EFF_RECO_STAT',
'MUON_EFF_RECO_SYS',
'MUON_EFF_RECO_STAT_LOWPT',
'MUON_EFF_RECO_SYS_LOWPT',
'MUON_EFF_ISO_STAT',
'MUON_EFF_ISO_SYS',
'MUON_EFF_BADMUON_STAT',
'MUON_EFF_BADMUON_SYS',
'MUON_EFF_TTVA_STAT',
'MUON_EFF_TTVA_SYS',
'FT_EFF_B_systematics',
'FT_EFF_C_systematics',
'FT_EFF_Light_systematics',
'FT_EFF_extrapolation',
'FT_EFF_extrapolation_from_charm',
'MET_SoftTrk_Scale',
'MET_SoftTrk_ResoPara',
'MET_SoftTrk_ResoPerp',
'jvtWeightJET_JvtEfficiency',
'pileupWeight']

In [6]:
signal_dict = {}

for n in unique_name:
    for names in root.keys():
        if n in str(names):
            if 'nosys' not in str(names).lower():
                if 'up' in str(names).lower():
                    if ('_'.join(str(names).split('_')[5:])).split('__')[0] in syst:
                        try:
                            signal_dict[n].append(str(names).split(";")[0].split("'")[1])
                        except:
                            signal_dict[n] = [str(names).split(";")[0].split("'")[1]]          

In [7]:
columns = ['met', 'mt', 'mbb', 'mct2',
        'mlb1', 'nJet30', 'lep1Pt', 'nBJet30_MV2c10',
           'genWeight','eventWeight', 'pileupWeight','leptonWeight','bTagWeight','jvtWeight',
        'trigMatch_metTrig', 'nLep_signal']

In [8]:
entrysteps=3000000
tot = 0

for folder in unique_name:
    dir = numpy_sig_syst_up + '{}'.format(folder)
    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)

    for name in signal_dict[folder]:

        events = uproot.open(data_folder)[name]
        array = events.lazyarray('met')

        print('lunghezza array', len(array))
        file_split = len(array)//entrysteps
        start_name_file = 0
        entrystart = start_name_file*entrysteps

        print(name)

        batches = events.iterate(columns, entrystart=entrystart,
                               entrysteps=entrysteps, 
                                   outputtype=pd.DataFrame)

        for ix in range(start_name_file, file_split+1):

            print(ix)    
            batch = next(batches)
            print('adding luminosity')
            batch['luminosity'] = 139000
            print(len(batch))

            batch = batch[batch['nLep_signal'].astype(int)==1]
            print('after signal {}'.format(len(batch)))

            batch = batch[batch['trigMatch_metTrig'].astype(int)==1]
            print('after trig {}'.format(len(batch)))

            batch = batch[((batch['nBJet30_MV2c10']>=1)&(batch['nBJet30_MV2c10']<4))]
            print('after bjet {}'.format(len(batch)))

            batch = batch[batch['met']>220]        
            print('after met {}'.format(len(batch)))

            batch = batch[batch['mt']>50]        
            print('after met {}'.format(len(batch)))

            batch = batch[((batch['mbb']>=100)&(batch['mbb']<=140))]  
            print('after mbb {}'.format(len(batch)))       

            batch = batch[batch['mct2']>100]
            print('after mct2 {}'.format(len(batch)))

            if len(batch) > 0:


                batch['weight'] = batch.apply(lambda row: row['genWeight']*row['eventWeight']*row['pileupWeight']*
                                     row['leptonWeight']*row['bTagWeight']*row['jvtWeight']*row['luminosity'], axis=1)

                batch_fin = batch.iloc[:,:8]

                batch_fin['weight'] = batch['weight']

                batch_fin = batch_fin[['met', 'mt', 'mbb', 'mct2',
                'mlb1','lep1Pt', 'nJet30', 'nBJet30_MV2c10', 'weight']]

                tot = tot + len(batch)
                print('tot = {}'.format(tot))
                print("\x1b[31m\"saving {}_{}""\x1b[0m".format(name,ix))
                np.save(numpy_sig_syst_up + '{}/{}.npy'.format(folder,name), batch_fin.values)


lunghezza array 874
C1N2_Wh_hbb_1000p0_0p0_EG_RESOLUTION_ALL__1up
0
adding luminosity
874
after signal 874
after trig 866
after bjet 866
after met 866
after met 866
after mbb 630
after mct2 619
tot = 619
"saving C1N2_Wh_hbb_1000p0_0p0_EG_RESOLUTION_ALL__1up_0
lunghezza array 872
C1N2_Wh_hbb_1000p0_0p0_EG_SCALE_AF2__1up
0
adding luminosity
872
after signal 872
after trig 864
after bjet 864
after met 864
after met 864
after mbb 629
after mct2 618
tot = 1237
"saving C1N2_Wh_hbb_1000p0_0p0_EG_SCALE_AF2__1up_0
lunghezza array 873
C1N2_Wh_hbb_1000p0_0p0_EG_SCALE_ALL__1up
0
adding luminosity
873
after signal 873
after trig 865
after bjet 865
after met 865
after met 865
after mbb 630
after mct2 619
tot = 1856
"saving C1N2_Wh_hbb_1000p0_0p0_EG_SCALE_ALL__1up_0
lunghezza array 872
C1N2_Wh_hbb_1000p0_0p0_JET_BJES_Response__1up
0
adding luminosity
872
after signal 872
after trig 864
after bjet 864
after met 864
after met 864
after mbb 629
after mct2 618
tot = 2474
"saving C1N2_Wh_hbb_1000p0_0p0_JE

lunghezza array 870
C1N2_Wh_hbb_1000p0_0p0_JET_JER_EffectiveNP_12restTerm__1up
0
adding luminosity
870
after signal 870
after trig 862
after bjet 862
after met 862
after met 862
after mbb 617
after mct2 606
tot = 18486
"saving C1N2_Wh_hbb_1000p0_0p0_JET_JER_EffectiveNP_12restTerm__1up_0
lunghezza array 867
C1N2_Wh_hbb_1000p0_0p0_JET_JER_EffectiveNP_1__1up
0
adding luminosity
867
after signal 867
after trig 860
after bjet 860
after met 860
after met 860
after mbb 612
after mct2 603
tot = 19089
"saving C1N2_Wh_hbb_1000p0_0p0_JET_JER_EffectiveNP_1__1up_0
lunghezza array 869
C1N2_Wh_hbb_1000p0_0p0_JET_JER_EffectiveNP_2__1up
0
adding luminosity
869
after signal 869
after trig 861
after bjet 861
after met 861
after met 861
after mbb 627
after mct2 617
tot = 19706
"saving C1N2_Wh_hbb_1000p0_0p0_JET_JER_EffectiveNP_2__1up_0
lunghezza array 868
C1N2_Wh_hbb_1000p0_0p0_JET_JER_EffectiveNP_3__1up
0
adding luminosity
868
after signal 868
after trig 861
after bjet 861
after met 861
after met 861
aft

lunghezza array 871
C1N2_Wh_hbb_1000p0_100p0_JET_EffectiveNP_Detector2__1up
0
adding luminosity
871
after signal 871
after trig 861
after bjet 861
after met 861
after met 861
after mbb 658
after mct2 646
tot = 37131
"saving C1N2_Wh_hbb_1000p0_100p0_JET_EffectiveNP_Detector2__1up_0
lunghezza array 871
C1N2_Wh_hbb_1000p0_100p0_JET_EffectiveNP_Mixed1__1up
0
adding luminosity
871
after signal 871
after trig 861
after bjet 861
after met 861
after met 861
after mbb 656
after mct2 644
tot = 37775
"saving C1N2_Wh_hbb_1000p0_100p0_JET_EffectiveNP_Mixed1__1up_0
lunghezza array 871
C1N2_Wh_hbb_1000p0_100p0_JET_EffectiveNP_Mixed2__1up
0
adding luminosity
871
after signal 871
after trig 861
after bjet 861
after met 861
after met 861
after mbb 658
after mct2 647
tot = 38422
"saving C1N2_Wh_hbb_1000p0_100p0_JET_EffectiveNP_Mixed2__1up_0
lunghezza array 871
C1N2_Wh_hbb_1000p0_100p0_JET_EffectiveNP_Mixed3__1up
0
adding luminosity
871
after signal 871
after trig 861
after bjet 861
after met 861
after me

lunghezza array 876
C1N2_Wh_hbb_1000p0_100p0_JET_JER_EffectiveNP_4__1up
0
adding luminosity
876
after signal 876
after trig 866
after bjet 866
after met 866
after met 866
after mbb 660
after mct2 648
tot = 55210
"saving C1N2_Wh_hbb_1000p0_100p0_JET_JER_EffectiveNP_4__1up_0
lunghezza array 874
C1N2_Wh_hbb_1000p0_100p0_JET_JER_EffectiveNP_5__1up
0
adding luminosity
874
after signal 874
after trig 864
after bjet 864
after met 864
after met 864
after mbb 658
after mct2 646
tot = 55856
"saving C1N2_Wh_hbb_1000p0_100p0_JET_JER_EffectiveNP_5__1up_0
lunghezza array 873
C1N2_Wh_hbb_1000p0_100p0_JET_JER_EffectiveNP_6__1up
0
adding luminosity
873
after signal 873
after trig 863
after bjet 863
after met 863
after met 863
after mbb 658
after mct2 646
tot = 56502
"saving C1N2_Wh_hbb_1000p0_100p0_JET_JER_EffectiveNP_6__1up_0
lunghezza array 875
C1N2_Wh_hbb_1000p0_100p0_JET_JER_EffectiveNP_7__1up
0
adding luminosity
875
after signal 875
after trig 865
after bjet 865
after met 865
after met 865
after m

lunghezza array 850
C1N2_Wh_hbb_1000p0_150p0_JET_EffectiveNP_Modelling1__1up
0
adding luminosity
850
after signal 850
after trig 845
after bjet 845
after met 845
after met 845
after mbb 648
after mct2 636
tot = 74475
"saving C1N2_Wh_hbb_1000p0_150p0_JET_EffectiveNP_Modelling1__1up_0
lunghezza array 852
C1N2_Wh_hbb_1000p0_150p0_JET_EffectiveNP_Modelling2__1up
0
adding luminosity
852
after signal 852
after trig 847
after bjet 847
after met 847
after met 847
after mbb 645
after mct2 635
tot = 75110
"saving C1N2_Wh_hbb_1000p0_150p0_JET_EffectiveNP_Modelling2__1up_0
lunghezza array 852
C1N2_Wh_hbb_1000p0_150p0_JET_EffectiveNP_Modelling3__1up
0
adding luminosity
852
after signal 852
after trig 847
after bjet 847
after met 847
after met 847
after mbb 643
after mct2 633
tot = 75743
"saving C1N2_Wh_hbb_1000p0_150p0_JET_EffectiveNP_Modelling3__1up_0
lunghezza array 852
C1N2_Wh_hbb_1000p0_150p0_JET_EffectiveNP_Modelling4__1up
0
adding luminosity
852
after signal 852
after trig 847
after bjet 847


...

lunghezza array 802
C1N2_Wh_hbb_900p0_350p0_JET_JER_EffectiveNP_5__1up
0
adding luminosity
802
after signal 802
after trig 793
after bjet 793
after met 793
after met 793
after mbb 582
after mct2 569
tot = 3264614
"saving C1N2_Wh_hbb_900p0_350p0_JET_JER_EffectiveNP_5__1up_0
lunghezza array 799
C1N2_Wh_hbb_900p0_350p0_JET_JER_EffectiveNP_6__1up
0
adding luminosity
799
after signal 799
after trig 790
after bjet 790
after met 790
after met 790
after mbb 585
after mct2 572
tot = 3265186
"saving C1N2_Wh_hbb_900p0_350p0_JET_JER_EffectiveNP_6__1up_0
lunghezza array 801
C1N2_Wh_hbb_900p0_350p0_JET_JER_EffectiveNP_7__1up
0
adding luminosity
801
after signal 801
after trig 792
after bjet 792
after met 792
after met 792
after mbb 584
after mct2 571
tot = 3265757
"saving C1N2_Wh_hbb_900p0_350p0_JET_JER_EffectiveNP_7__1up_0
lunghezza array 797
C1N2_Wh_hbb_900p0_350p0_JET_JER_EffectiveNP_8__1up
0
adding luminosity
797
after signal 797
after trig 788
after bjet 788
after met 788
after met 788
after mb

lunghezza array 771
C1N2_Wh_hbb_900p0_400p0_JET_EffectiveNP_Modelling2__1up
0
adding luminosity
771
after signal 771
after trig 763
after bjet 763
after met 763
after met 763
after mbb 567
after mct2 548
tot = 3281516
"saving C1N2_Wh_hbb_900p0_400p0_JET_EffectiveNP_Modelling2__1up_0
lunghezza array 771
C1N2_Wh_hbb_900p0_400p0_JET_EffectiveNP_Modelling3__1up
0
adding luminosity
771
after signal 771
after trig 763
after bjet 763
after met 763
after met 763
after mbb 568
after mct2 549
tot = 3282065
"saving C1N2_Wh_hbb_900p0_400p0_JET_EffectiveNP_Modelling3__1up_0
lunghezza array 771
C1N2_Wh_hbb_900p0_400p0_JET_EffectiveNP_Modelling4__1up
0
adding luminosity
771
after signal 771
after trig 763
after bjet 763
after met 763
after met 763
after mbb 567
after mct2 548
tot = 3282613
"saving C1N2_Wh_hbb_900p0_400p0_JET_EffectiveNP_Modelling4__1up_0
lunghezza array 772
C1N2_Wh_hbb_900p0_400p0_JET_EffectiveNP_Statistical1__1up
0
adding luminosity
772
after signal 772
after trig 764
after bjet 764

lunghezza array 777
C1N2_Wh_hbb_900p0_400p0_JET_JER_EffectiveNP_9__1up
0
adding luminosity
777
after signal 777
after trig 769
after bjet 769
after met 769
after met 769
after mbb 572
after mct2 552
tot = 3296888
"saving C1N2_Wh_hbb_900p0_400p0_JET_JER_EffectiveNP_9__1up_0
lunghezza array 772
C1N2_Wh_hbb_900p0_400p0_JET_Pileup_OffsetMu__1down
0
adding luminosity
772
after signal 772
after trig 763
after bjet 763
after met 763
after met 763
after mbb 571
after mct2 553
tot = 3297441
"saving C1N2_Wh_hbb_900p0_400p0_JET_Pileup_OffsetMu__1down_0
lunghezza array 770
C1N2_Wh_hbb_900p0_400p0_JET_Pileup_OffsetMu__1up
0
adding luminosity
770
after signal 770
after trig 762
after bjet 762
after met 762
after met 762
after mbb 565
after mct2 546
tot = 3297987
"saving C1N2_Wh_hbb_900p0_400p0_JET_Pileup_OffsetMu__1up_0
lunghezza array 768
C1N2_Wh_hbb_900p0_400p0_JET_Pileup_OffsetNPV__1down
0
adding luminosity
768
after signal 768
after trig 759
after bjet 759
after met 759
after met 759
after mbb 5

lunghezza array 860
C1N2_Wh_hbb_900p0_50p0_JET_EffectiveNP_Statistical2__1up
0
adding luminosity
860
after signal 860
after trig 851
after bjet 851
after met 851
after met 851
after mbb 650
after mct2 631
tot = 3314590
"saving C1N2_Wh_hbb_900p0_50p0_JET_EffectiveNP_Statistical2__1up_0
lunghezza array 859
C1N2_Wh_hbb_900p0_50p0_JET_EffectiveNP_Statistical3__1up
0
adding luminosity
859
after signal 859
after trig 850
after bjet 850
after met 850
after met 850
after mbb 649
after mct2 630
tot = 3315220
"saving C1N2_Wh_hbb_900p0_50p0_JET_EffectiveNP_Statistical3__1up_0
lunghezza array 860
C1N2_Wh_hbb_900p0_50p0_JET_EffectiveNP_Statistical4__1up
0
adding luminosity
860
after signal 860
after trig 851
after bjet 851
after met 851
after met 851
after mbb 650
after mct2 631
tot = 3315851
"saving C1N2_Wh_hbb_900p0_50p0_JET_EffectiveNP_Statistical4__1up_0
lunghezza array 860
C1N2_Wh_hbb_900p0_50p0_JET_EffectiveNP_Statistical5__1up
0
adding luminosity
860
after signal 860
after trig 851
after bje

lunghezza array 859
C1N2_Wh_hbb_900p0_50p0_JET_Pileup_PtTerm__1down
0
adding luminosity
859
after signal 859
after trig 850
after bjet 850
after met 850
after met 850
after mbb 648
after mct2 629
tot = 3332872
"saving C1N2_Wh_hbb_900p0_50p0_JET_Pileup_PtTerm__1down_0
lunghezza array 861
C1N2_Wh_hbb_900p0_50p0_JET_Pileup_PtTerm__1up
0
adding luminosity
861
after signal 861
after trig 852
after bjet 852
after met 852
after met 852
after mbb 648
after mct2 631
tot = 3333503
"saving C1N2_Wh_hbb_900p0_50p0_JET_Pileup_PtTerm__1up_0
lunghezza array 863
C1N2_Wh_hbb_900p0_50p0_JET_Pileup_RhoTopology__1down
0
adding luminosity
863
after signal 863
after trig 854
after bjet 854
after met 854
after met 854
after mbb 648
after mct2 630
tot = 3334133
"saving C1N2_Wh_hbb_900p0_50p0_JET_Pileup_RhoTopology__1down_0
lunghezza array 859
C1N2_Wh_hbb_900p0_50p0_JET_Pileup_RhoTopology__1up
0
adding luminosity
859
after signal 859
after trig 850
after bjet 850
after met 850
after met 850
after mbb 651
after 